In [1]:
%%writefile query.hql

ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar;
ADD JAR /opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar;

USE stackoverflow_;

WITH tags_2009 as 
        (SELECT year, tag, count(1) as popularity_tag, RANK() OVER (PARTITION BY year ORDER BY count(1) DESC) as rank_t
        FROM posts
        LATERAL VIEW explode(tags) lateral_table AS tag
        WHERE year = '2009' AND post_type_id = '1'
        GROUP BY year, tag),
    tags_2016 as 
        (SELECT year, tag, count(1) as popularity_tag, RANK() OVER (PARTITION BY year ORDER BY count(1) DESC) as rank_t
        FROM posts
        LATERAL VIEW explode(tags) lateral_table AS tag
        WHERE year = '2016' AND post_type_id = '1'
        GROUP BY year, tag)
SELECT a.tag, b.rank_t, a.rank_t, b.popularity_tag, a.popularity_tag 
    FROM (SELECT tag, rank_t, popularity_tag from tags_2009) a 
    JOIN (SELECT tag, rank_t, popularity_tag from tags_2016) b 
    ON a.tag = b.tag
    ORDER BY b.popularity_tag DESC
    LIMIT 10;

Writing query.hql


In [ ]:
! hive -f query.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-contrib.jar]
Added [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar] to class path
Added resources: [/opt/cloudera/parcels/CDH/lib/hive/lib/hive-serde.jar]
OK
Time taken: 1.04 seconds
Query ID = jovyan_20180716052626_3b93f474-b22d-452c-85d0-e4ff006bdf23
Total jobs = 8
Launching Job 1 out of 8
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1531693512718_0042, Tracking URL = http://b5a379